# Multi-Scenario MORDM

Multi-scenario MORMD is an extension of normal MORDM to better include robustness considerations within the search phase. It starts from the scenario discovery results resulting from MORDM. Next, from the experiments within this box, a set of scenarios is selected. 


In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

from ema_workbench import (Model, CategoricalParameter,
                           ScalarOutcome, IntegerParameter, RealParameter,
                           MultiprocessingEvaluator, Policy, Scenario)

from dike_model_function import DikeNetwork  # @UnresolvedImport


def sum_over(*args):
    return sum(args)

from ema_workbench import (Model, )

from ema_workbench.em_framework.evaluators import perform_experiments
from ema_workbench.em_framework.samplers import sample_uncertainties
from ema_workbench.util import ema_logging
import time

from problem_formulation import get_model_for_problem_formulation


ema_logging.log_to_stderr(ema_logging.INFO)

#choose problem formulation number, between 0-5
#each problem formulation has its own list of outcomes
dike_model, planning_steps = get_model_for_problem_formulation(5)

In [2]:
uncertainties = dike_model.uncertainties
levers = dike_model.levers 

import copy
uncertainties = copy.deepcopy(dike_model.uncertainties)
levers = copy.deepcopy(dike_model.levers)

#running the model through EMA workbench
from ema_workbench import (MultiprocessingEvaluator, ema_logging,
                           perform_experiments, SequentialEvaluator)
ema_logging.log_to_stderr(ema_logging.INFO)
 
with MultiprocessingEvaluator(dike_model) as evaluator:
    experiments, outcomes = evaluator.perform_experiments(scenarios=10, policies = 3)

or import already available experiments

In [3]:
import pandas as pd
#from ema_workbench import load_results

#experiments, outcomes = load_results('mordm_42.tar.gz')
#y = outcomes['utility'] < 0.35
experiments = pd.read_csv('outcomes/Experiments_from_Exploration_1000scenarios_100Policies_pf5.csv')
outcomes = pd.read_csv('outcomes/Outcomes_from_Exploration_1000scenarios_100Policies_pf5.csv')
outcomes.head()

,Unnamed: 0,A.1_Expected Annual Damage 0,A.1_Dike Investment Costs 0,A.1_Expected Number of Deaths 0,A.2_Expected Annual Damage 0,A.2_Dike Investment Costs 0,A.2_Expected Number of Deaths 0,A.3_Expected Annual Damage 0,A.3_Dike Investment Costs 0,A.3_Expected Number of Deaths 0,...,A.3_Dike Investment Costs 2,A.3_Expected Number of Deaths 2,A.4_Expected Annual Damage 2,A.4_Dike Investment Costs 2,A.4_Expected Number of Deaths 2,A.5_Expected Annual Damage 2,A.5_Dike Investment Costs 2,A.5_Expected Number of Deaths 2,RfR Total Costs 2,Expected Evacuation Costs 2
0,0,0.0,6.071731e+07,0.0,0.0,9.952540e+07,0.0,0.000000e+00,2.419803e+07,0.000000,...,5.876762e+07,0.0,0.0,1.309366e+07,0.0,0.0,4.542428e+07,0.0,679700000.0,0.0
1,1,0.0,6.071731e+07,0.0,0.0,9.952540e+07,0.0,3.036115e+06,2.419803e+07,0.001489,...,5.876762e+07,0.0,0.0,1.309366e+07,0.0,0.0,4.542428e+07,0.0,679700000.0,0.0
2,2,0.0,6.071731e+07,0.0,0.0,9.952540e+07,0.0,1.740958e+08,2.419803e+07,0.128044,...,5.876762e+07,0.0,0.0,1.309366e+07,0.0,0.0,4.542428e+07,0.0,679700000.0,0.0
3,3,0.0,6.071731e+07,0.0,0.0,9.952540e+07,0.0,2.703954e+07,2.419803e+07,0.013140,...,5.876762e+07,0.0,0.0,1.309366e+07,0.0,0.0,4.542428e+07,0.0,679700000.0,0.0
4,4,0.0,6.071731e+07,0.0,0.0,9.952540e+07,0.0,0.000000e+00,2.419803e+07,0.000000,...,5.876762e+07,0.0,0.0,1.309366e+07,0.0,0.0,4.542428e+07,0.0,679700000.0,0.0


In [4]:
experiments_DF = pd.DataFrame(experiments)
experiments_DF = experiments_DF.iloc[:, 1:]
experiments_DF.head()

,A.0_ID flood wave shape,A.1_Bmax,A.1_Brate,A.1_pfail,A.2_Bmax,A.2_Brate,A.2_pfail,A.3_Bmax,A.3_Brate,A.3_pfail,...,A.4_DikeIncrease 0,A.4_DikeIncrease 1,A.4_DikeIncrease 2,A.5_DikeIncrease 0,A.5_DikeIncrease 1,A.5_DikeIncrease 2,EWS_DaysToThreat,scenario,policy,model
0,126.0,223.622426,1.0,0.871722,218.831042,1.0,0.705490,237.027027,10.0,0.852197,...,5.0,0.0,5.0,4.0,8.0,3.0,1.0,0,0,dikesnet
1,79.0,298.108524,1.0,0.100861,232.253542,1.0,0.104278,33.997448,1.5,0.316080,...,5.0,0.0,5.0,4.0,8.0,3.0,1.0,1,0,dikesnet
2,80.0,275.541284,10.0,0.200497,229.990714,1.5,0.055838,114.045936,1.5,0.031793,...,5.0,0.0,5.0,4.0,8.0,3.0,1.0,2,0,dikesnet
3,15.0,200.546816,1.0,0.000321,281.785818,1.0,0.018157,162.548088,1.0,0.153352,...,5.0,0.0,5.0,4.0,8.0,3.0,1.0,3,0,dikesnet
4,120.0,312.688358,1.5,0.202780,334.804942,1.5,0.047327,178.394116,1.5,0.513973,...,5.0,0.0,5.0,4.0,8.0,3.0,1.0,4,0,dikesnet


there are many ways in which you can make an informed selection. Examples include
* --> pick best and worst for each outcome of interest
* maximize diversity following Eker & Kwakkel
* visually selected them

## best case, worst case

In [5]:
outcomes_DF = pd.DataFrame(outcomes)
outcomes_DF = outcomes_DF.iloc[:, 1:]
outcomes_DF.head()

,A.1_Expected Annual Damage 0,A.1_Dike Investment Costs 0,A.1_Expected Number of Deaths 0,A.2_Expected Annual Damage 0,A.2_Dike Investment Costs 0,A.2_Expected Number of Deaths 0,A.3_Expected Annual Damage 0,A.3_Dike Investment Costs 0,A.3_Expected Number of Deaths 0,A.4_Expected Annual Damage 0,...,A.3_Dike Investment Costs 2,A.3_Expected Number of Deaths 2,A.4_Expected Annual Damage 2,A.4_Dike Investment Costs 2,A.4_Expected Number of Deaths 2,A.5_Expected Annual Damage 2,A.5_Dike Investment Costs 2,A.5_Expected Number of Deaths 2,RfR Total Costs 2,Expected Evacuation Costs 2
0,0.0,6.071731e+07,0.0,0.0,9.952540e+07,0.0,0.000000e+00,2.419803e+07,0.000000,0.0,...,5.876762e+07,0.0,0.0,1.309366e+07,0.0,0.0,4.542428e+07,0.0,679700000.0,0.0
1,0.0,6.071731e+07,0.0,0.0,9.952540e+07,0.0,3.036115e+06,2.419803e+07,0.001489,0.0,...,5.876762e+07,0.0,0.0,1.309366e+07,0.0,0.0,4.542428e+07,0.0,679700000.0,0.0
2,0.0,6.071731e+07,0.0,0.0,9.952540e+07,0.0,1.740958e+08,2.419803e+07,0.128044,0.0,...,5.876762e+07,0.0,0.0,1.309366e+07,0.0,0.0,4.542428e+07,0.0,679700000.0,0.0
3,0.0,6.071731e+07,0.0,0.0,9.952540e+07,0.0,2.703954e+07,2.419803e+07,0.013140,0.0,...,5.876762e+07,0.0,0.0,1.309366e+07,0.0,0.0,4.542428e+07,0.0,679700000.0,0.0
4,0.0,6.071731e+07,0.0,0.0,9.952540e+07,0.0,0.000000e+00,2.419803e+07,0.000000,0.0,...,5.876762e+07,0.0,0.0,1.309366e+07,0.0,0.0,4.542428e+07,0.0,679700000.0,0.0


In [6]:
outcomes_DF['A.5_Total Expected Number of Deaths'] = outcomes_DF['A.5_Expected Number of Deaths 0'] + outcomes_DF['A.5_Expected Number of Deaths 1'] + outcomes_DF['A.5_Expected Number of Deaths 2']
outcomes_DF.head()

,A.1_Expected Annual Damage 0,A.1_Dike Investment Costs 0,A.1_Expected Number of Deaths 0,A.2_Expected Annual Damage 0,A.2_Dike Investment Costs 0,A.2_Expected Number of Deaths 0,A.3_Expected Annual Damage 0,A.3_Dike Investment Costs 0,A.3_Expected Number of Deaths 0,A.4_Expected Annual Damage 0,...,A.3_Expected Number of Deaths 2,A.4_Expected Annual Damage 2,A.4_Dike Investment Costs 2,A.4_Expected Number of Deaths 2,A.5_Expected Annual Damage 2,A.5_Dike Investment Costs 2,A.5_Expected Number of Deaths 2,RfR Total Costs 2,Expected Evacuation Costs 2,A.5_Total Expected Number of Deaths
0,0.0,6.071731e+07,0.0,0.0,9.952540e+07,0.0,0.000000e+00,2.419803e+07,0.000000,0.0,...,0.0,0.0,1.309366e+07,0.0,0.0,4.542428e+07,0.0,679700000.0,0.0,0.000253
1,0.0,6.071731e+07,0.0,0.0,9.952540e+07,0.0,3.036115e+06,2.419803e+07,0.001489,0.0,...,0.0,0.0,1.309366e+07,0.0,0.0,4.542428e+07,0.0,679700000.0,0.0,0.000000
2,0.0,6.071731e+07,0.0,0.0,9.952540e+07,0.0,1.740958e+08,2.419803e+07,0.128044,0.0,...,0.0,0.0,1.309366e+07,0.0,0.0,4.542428e+07,0.0,679700000.0,0.0,0.000000
3,0.0,6.071731e+07,0.0,0.0,9.952540e+07,0.0,2.703954e+07,2.419803e+07,0.013140,0.0,...,0.0,0.0,1.309366e+07,0.0,0.0,4.542428e+07,0.0,679700000.0,0.0,0.000000
4,0.0,6.071731e+07,0.0,0.0,9.952540e+07,0.0,0.000000e+00,2.419803e+07,0.000000,0.0,...,0.0,0.0,1.309366e+07,0.0,0.0,4.542428e+07,0.0,679700000.0,0.0,0.000000


In [7]:
outcomes_DF.describe()

,A.1_Expected Annual Damage 0,A.1_Dike Investment Costs 0,A.1_Expected Number of Deaths 0,A.2_Expected Annual Damage 0,A.2_Dike Investment Costs 0,A.2_Expected Number of Deaths 0,A.3_Expected Annual Damage 0,A.3_Dike Investment Costs 0,A.3_Expected Number of Deaths 0,A.4_Expected Annual Damage 0,...,A.3_Expected Number of Deaths 2,A.4_Expected Annual Damage 2,A.4_Dike Investment Costs 2,A.4_Expected Number of Deaths 2,A.5_Expected Annual Damage 2,A.5_Dike Investment Costs 2,A.5_Expected Number of Deaths 2,RfR Total Costs 2,Expected Evacuation Costs 2,A.5_Total Expected Number of Deaths
count,1.000000e+05,1.000000e+05,100000.000000,1.000000e+05,1.000000e+05,100000.000000,1.000000e+05,1.000000e+05,100000.000000,1.000000e+05,...,100000.000000,1.000000e+05,1.000000e+05,100000.000000,1.000000e+05,1.000000e+05,100000.000000,1.000000e+05,100000.000000,100000.000000
mean,9.254968e+06,5.440302e+07,0.001996,9.177039e+06,5.915342e+07,0.002956,1.277003e+07,2.813166e+07,0.008003,1.131574e+06,...,0.000421,5.024640e+04,1.607228e+07,0.000014,5.406340e+05,5.163506e+07,0.000128,3.552000e+08,77.882943,0.003768
std,8.317559e+07,2.557792e+07,0.022412,3.635736e+07,2.634051e+07,0.015882,6.852759e+07,1.144721e+07,0.053934,4.620627e+06,...,0.008356,8.467630e+05,8.386790e+06,0.000236,6.279000e+06,2.472235e+07,0.001488,1.838996e+08,830.296883,0.020274
min,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,...,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000
25%,0.000000e+00,3.725520e+07,0.000000,0.000000e+00,4.281842e+07,0.000000,0.000000e+00,2.216782e+07,0.000000,0.000000e+00,...,0.000000,0.000000e+00,1.061066e+07,0.000000,0.000000e+00,3.726983e+07,0.000000,2.178000e+08,0.000000,0.000000
50%,0.000000e+00,5.397251e+07,0.000000,0.000000e+00,5.941010e+07,0.000000,0.000000e+00,2.879840e+07,0.000000,0.000000e+00,...,0.000000,0.000000e+00,1.520173e+07,0.000000,0.000000e+00,4.978985e+07,0.000000,3.697000e+08,0.000000,0.000000
75%,0.000000e+00,7.629909e+07,0.000000,1.609640e+06,8.127741e+07,0.000343,0.000000e+00,3.728504e+07,0.000000,0.000000e+00,...,0.000000,0.000000e+00,2.254184e+07,0.000000,0.000000e+00,6.737315e+07,0.000000,4.956000e+08,0.000000,0.000000
max,1.202386e+09,9.510733e+07,0.588712,4.151095e+08,9.952540e+07,0.267862,8.516525e+08,4.421502e+07,1.075712,6.674499e+07,...,0.964167,4.099444e+07,3.859670e+07,0.012702,3.145494e+08,1.101545e+08,0.071303,6.797000e+08,24724.639475,0.645636


In [8]:
y = outcomes_DF['A.5_Total Expected Number of Deaths'] > 1/100000 # too many, just for the sake of demonstration

data = pd.DataFrame({k:v[y] for k,v in outcomes_DF.items()})
all_data = pd.DataFrame({k:v for k,v in outcomes_DF.items()})

In [9]:
ylist = y.tolist()

# check how many truw in list
def count(lst): 
    return sum(bool(x) for x in lst)      
print(count(ylist))

21932


#way too much data --> focus on some 
from ema_workbench.analysis import parcoords
import matplotlib.pyplot as plt

limits = parcoords.get_limits(all_data)

axes = parcoords.ParallelAxes(limits)

axes.plot(all_data, color='lightgrey')
axes.plot(data, color='blue')
plt.show()

In [10]:
data.head()

,A.1_Expected Annual Damage 0,A.1_Dike Investment Costs 0,A.1_Expected Number of Deaths 0,A.2_Expected Annual Damage 0,A.2_Dike Investment Costs 0,A.2_Expected Number of Deaths 0,A.3_Expected Annual Damage 0,A.3_Dike Investment Costs 0,A.3_Expected Number of Deaths 0,A.4_Expected Annual Damage 0,...,A.3_Expected Number of Deaths 2,A.4_Expected Annual Damage 2,A.4_Dike Investment Costs 2,A.4_Expected Number of Deaths 2,A.5_Expected Annual Damage 2,A.5_Dike Investment Costs 2,A.5_Expected Number of Deaths 2,RfR Total Costs 2,Expected Evacuation Costs 2,A.5_Total Expected Number of Deaths
0,0.0,6.071731e+07,0.0,0.0,9.952540e+07,0.0,0.0,2.419803e+07,0.0,0.0,...,0.0,0.0,1.309366e+07,0.0,0.0,4.542428e+07,0.0,679700000.0,0.0,0.000253
10,0.0,6.071731e+07,0.0,0.0,9.952540e+07,0.0,0.0,2.419803e+07,0.0,0.0,...,0.0,0.0,1.309366e+07,0.0,0.0,4.542428e+07,0.0,679700000.0,0.0,0.003089
14,0.0,6.071731e+07,0.0,0.0,9.952540e+07,0.0,0.0,2.419803e+07,0.0,0.0,...,0.0,0.0,1.309366e+07,0.0,0.0,4.542428e+07,0.0,679700000.0,0.0,0.011565
36,0.0,6.071731e+07,0.0,0.0,9.952540e+07,0.0,0.0,2.419803e+07,0.0,0.0,...,0.0,0.0,1.309366e+07,0.0,0.0,4.542428e+07,0.0,679700000.0,0.0,0.003203
45,0.0,6.071731e+07,0.0,0.0,9.952540e+07,0.0,0.0,2.419803e+07,0.0,0.0,...,0.0,0.0,1.309366e+07,0.0,0.0,4.542428e+07,0.0,679700000.0,0.0,0.003501


In [11]:
data.describe()

,A.1_Expected Annual Damage 0,A.1_Dike Investment Costs 0,A.1_Expected Number of Deaths 0,A.2_Expected Annual Damage 0,A.2_Dike Investment Costs 0,A.2_Expected Number of Deaths 0,A.3_Expected Annual Damage 0,A.3_Dike Investment Costs 0,A.3_Expected Number of Deaths 0,A.4_Expected Annual Damage 0,...,A.3_Expected Number of Deaths 2,A.4_Expected Annual Damage 2,A.4_Dike Investment Costs 2,A.4_Expected Number of Deaths 2,A.5_Expected Annual Damage 2,A.5_Dike Investment Costs 2,A.5_Expected Number of Deaths 2,RfR Total Costs 2,Expected Evacuation Costs 2,A.5_Total Expected Number of Deaths
count,2.193200e+04,2.193200e+04,21932.000000,2.193200e+04,2.193200e+04,21932.000000,2.193200e+04,2.193200e+04,21932.000000,2.193200e+04,...,21932.000000,2.193200e+04,2.193200e+04,2.193200e+04,2.193200e+04,2.193200e+04,21932.000000,2.193200e+04,21932.000000,21932.000000
mean,4.123012e+06,5.398613e+07,0.001170,3.409467e+06,6.251669e+07,0.002028,2.566886e+06,3.208294e+07,0.002037,6.083869e+05,...,0.000050,9.988606e+02,1.589294e+07,2.774555e-07,2.465047e+06,4.243922e+07,0.000583,3.432699e+08,124.694316,0.017181
std,5.418890e+07,2.395805e+07,0.017837,2.080638e+07,2.320132e+07,0.015608,2.260298e+07,9.719431e+06,0.022326,2.970664e+06,...,0.001056,4.332730e+04,8.603655e+06,1.198320e-05,1.322975e+07,1.804208e+07,0.003134,1.754705e+08,743.772045,0.040543
min,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,...,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000049
25%,0.000000e+00,4.229151e+07,0.000000,0.000000e+00,4.784270e+07,0.000000,0.000000e+00,2.640338e+07,0.000000,0.000000e+00,...,0.000000,0.000000e+00,1.061066e+07,0.000000e+00,0.000000e+00,3.471760e+07,0.000000,2.365000e+08,0.000000,0.001211
50%,0.000000e+00,5.397251e+07,0.000000,0.000000e+00,5.941010e+07,0.000000,0.000000e+00,3.422156e+07,0.000000,0.000000e+00,...,0.000000,0.000000e+00,1.646804e+07,0.000000e+00,0.000000e+00,4.419351e+07,0.000000,3.331000e+08,0.000000,0.004349
75%,0.000000e+00,7.629909e+07,0.000000,0.000000e+00,8.127741e+07,0.000000,0.000000e+00,4.060905e+07,0.000000,0.000000e+00,...,0.000000,0.000000e+00,2.207342e+07,0.000000e+00,0.000000e+00,5.183238e+07,0.000000,4.739000e+08,0.000000,0.014840
max,1.202386e+09,9.510733e+07,0.588311,4.151095e+08,9.952540e+07,0.267862,8.084248e+08,4.421502e+07,1.075712,6.265503e+07,...,0.062706,3.345441e+06,3.859670e+07,7.626032e-04,3.145494e+08,1.029953e+08,0.071303,6.797000e+08,21927.447177,0.645636


in the next step, we select worst case scenarios

In [12]:
#take worst value in each column
maxrow = data.idxmax()
maxrow = maxrow.tolist()
# take best value in each column
minrow = data.idxmin()
minrow = minrow.tolist()

In [13]:
# take 20 worst values for total deaths
row_high_death = data.loc[data['A.5_Total Expected Number of Deaths'] > 0.5]
row_high_death = row_high_death.index
row_high_death = row_high_death.tolist()
row_high_death

[21462,
 21587,
 21682,
 21731,
 21780,
 21819,
 64266,
 64323,
 64462,
 64521,
 64540,
 64587,
 64682,
 64780,
 64819,
 64882]

In [14]:
# also all we need are the uncertainty columns --> for maxrows
selected = experiments_DF.loc[row_high_death, experiments_DF.columns[0:-3]]
selected

,A.0_ID flood wave shape,A.1_Bmax,A.1_Brate,A.1_pfail,A.2_Bmax,A.2_Brate,A.2_pfail,A.3_Bmax,A.3_Brate,A.3_pfail,...,A.3_DikeIncrease 0,A.3_DikeIncrease 1,A.3_DikeIncrease 2,A.4_DikeIncrease 0,A.4_DikeIncrease 1,A.4_DikeIncrease 2,A.5_DikeIncrease 0,A.5_DikeIncrease 1,A.5_DikeIncrease 2,EWS_DaysToThreat
21462,75.0,110.135623,1.0,0.509537,226.629336,10.0,0.367084,85.981938,1.5,0.816194,...,3.0,0.0,10.0,2.0,8.0,5.0,3.0,0.0,10.0,0.0
21587,104.0,96.562103,1.0,0.765842,198.747874,1.0,0.940758,115.717538,1.5,0.743451,...,3.0,0.0,10.0,2.0,8.0,5.0,3.0,0.0,10.0,0.0
21682,22.0,247.631754,1.0,0.814604,108.813284,1.0,0.513680,245.721256,10.0,0.450826,...,3.0,0.0,10.0,2.0,8.0,5.0,3.0,0.0,10.0,0.0
21731,41.0,91.096511,10.0,0.001297,300.011472,1.0,0.096849,331.126590,1.5,0.701429,...,3.0,0.0,10.0,2.0,8.0,5.0,3.0,0.0,10.0,0.0
21780,20.0,133.292348,10.0,0.635681,282.862724,10.0,0.242201,123.826512,1.0,0.413829,...,3.0,0.0,10.0,2.0,8.0,5.0,3.0,0.0,10.0,0.0
21819,5.0,259.418962,10.0,0.783973,50.301659,1.0,0.518480,178.574143,10.0,0.520274,...,3.0,0.0,10.0,2.0,8.0,5.0,3.0,0.0,10.0,0.0
64266,112.0,193.150509,10.0,0.990240,80.042670,10.0,0.317445,53.670594,1.0,0.873124,...,4.0,8.0,3.0,10.0,5.0,2.0,1.0,3.0,6.0,0.0
64323,126.0,269.639014,1.0,0.351077,322.179020,10.0,0.771631,341.247871,10.0,0.299240,...,4.0,8.0,3.0,10.0,5.0,2.0,1.0,3.0,6.0,0.0
64462,75.0,110.135623,1.0,0.509537,226.629336,10.0,0.367084,85.981938,1.5,0.816194,...,4.0,8.0,3.0,10.0,5.0,2.0,1.0,3.0,6.0,0.0
64521,67.0,235.418596,1.5,0.830730,37.343842,10.0,0.923498,319.598667,10.0,0.978417,...,4.0,8.0,3.0,10.0,5.0,2.0,1.0,3.0,6.0,0.0


## Search for each scenario

For each of the four selected scenarios, use many-objective optimization to find a pareto approximate set using the same approach as for assignment 8. Remember to check for convergence (and time permitting, seed analysis), and be careful in what epsilon values to use (not to coarse, not too small). 

Store the resulting set of pareto solutions in a smart way for subsequent analysis.


Since we have to do the same thing for each scenario, it is convenient to wrap this in a function we can call with the scenario under which we want to optimize. 

In [15]:
from ema_workbench import Scenario

scenarios = [Scenario(f"{index}", **row) for index, row in selected.iterrows()]
#scenarios

In [16]:
from ema_workbench import MultiprocessingEvaluator, ema_logging, SequentialEvaluator
from ema_workbench.em_framework.evaluators import BaseEvaluator

from ema_workbench.em_framework.optimization import (HyperVolume,
                                                     EpsilonProgress)

In [17]:
ema_logging.log_to_stderr(ema_logging.INFO)

def optimize(scenario, nfe, model, converge_metrics, epsilons):


    with SequentialEvaluator(model) as evaluator:
        results, convergence = evaluator.optimize(nfe=nfe, searchover='levers',
                                     convergence=convergence_metrics,
                                     epsilons=epsilons,
                                     reference=scenario)
    return results, convergence


results = []
for scenario in scenarios:
    convergence_metrics = [#HyperVolume(minimum=[0,0,0,0], maximum=[3, 2,1.01,1.01]),
                           EpsilonProgress()]
    epsilons = [0.1,]*len(dike_model.outcomes) #0.1
    
    results.append(optimize(scenario, 1e2, dike_model, convergence_metrics, epsilons)) #1e4 --> list index out of range



[MainProcess/INFO] generation 0: 0/100 nfe


ValueError: cannot convert float NaN to integer

In [22]:
ema_logging.log_to_stderr(ema_logging.INFO)

model, steps = get_model_for_problem_formulation(5)

reference_values = {'Bmax': 175, 'Brate': 1.5, 'pfail': 0.5,
                    'discount rate 0': 3.5, 'discount rate 1': 3.5,
                    'discount rate 2': 3.5,
                    'ID flood wave shape': 4}
scen1 = {}

for key in dike_model.uncertainties:
    name_split = key.name.split('_')

    if len(name_split) == 1:
        scen1.update({key.name: reference_values[key.name]})

    else:
        scen1.update({key.name: reference_values[name_split[1]]})

ref_scenario = Scenario('reference', **scen1)

convergence_metrics = [EpsilonProgress()]

epsilon = [1e3,] * len(dike_model.outcomes)

nfe = 10 # proof of principle only, way to low for actual use

for scenario in ref_scenario:
    with SequentialEvaluator(dike_model) as evaluator:
        results, convergence = evaluator.optimize(nfe=nfe, searchover='levers', epsilons=epsilon, 
                                                  convergence=convergence_metrics, reference=scenarios)



AssertionError: 

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2, sharex=True)
fig, ax1 = plt.subplots(ncols=1)
ax1.plot(convergence.epsilon_progress)
ax1.set_xlabel('nr. of generations')
ax1.set_ylabel('$\epsilon$ progress')
sns.despine()

In [ ]:
results[0]

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2)
for i, (_, convergence) in enumerate(results):
    ax1.plot(convergence.nfe, convergence.hypervolume, label=f'scenario {i}')
    ax2.plot(convergence.nfe, convergence.epsilon_progress, label=f'scenario {i}')

#ax1.set_ylabel('hypervolume')
ax1.set_xlabel('nfe')
ax2.set_ylabel('$\epsilon$ progress')
ax2.set_xlabel('nfe')
fig.legend()
plt.show()

## From here on kwakkel's solution

below is a quick visualization of the trade-offs found for each scenario. It does reveal how optimizing under differen scenarios reveals different trade-offs

Note that this is in a way a misleading figure because the scenario under which each of these sets of solutions is being evaluated is not the same.

In [ ]:
from ema_workbench.analysis import parcoords

colors = iter(sns.color_palette())

#check
data = results[0][0].iloc[:, 5::]
limits = parcoords.get_limits(data)

#check
limits.loc[0, ['inertia', 'reliability']] = 1
limits.loc[0, 'max_P'] = 4 # max over results based on quick inspection not shown here
limits.loc[0, 'utility'] = 1 # max over results based on quick inspection not shown here
limits.loc[1, :] = 0
paraxes = parcoords.ParallelAxes(limits)


for i, (result, _) in enumerate(results):
    color = next(colors)
    data = result.iloc[:, 5::]
    paraxes.plot(data, label=f'scenario {i}', color=color)

paraxes.legend()
plt.show()


## Re-evaluate under deep uncertainty

Combine the pareto set of solutions found for each scenario. Next, turn each solution into a policy object. If you have a very large number of policies, you can choose to down sample your policies in some reasoned way (*e.g.*, picking min and max on each objective, slicing across the pareto front with a particular step size). As a rule of thumb, try to limit the set of policies to at most 50. 

Re-evaluate the combined set of solutions over 1000 scenarios sampled using LHS.


In [ ]:
from ema_workbench import Policy

policies = []
for i, (result, _) in enumerate(results):
    result = result.iloc[:, 0:5]
    for j, row in result.iterrows():
        policy = Policy(f'scenario {i} option {j}', **row.to_dict())
        policies.append(policy)

In [ ]:
with MultiprocessingEvaluator(model) as evaluator:
    reeevaluation_results = evaluator.perform_experiments(1000, policies=policies)

Calculate both the maximum regret, and the domain criterion using the values provided in [Bartholomew and Kwakkel (2020)](https://doi.org/10.1016/j.envsoft.2020.104699). Ignore the max_P objective.

visualize the results in parallel coordinate plot. 

Are there any promising compromise solutions which balance performance in both the reference scenarios as well as in terms of their robustness?


In [ ]:
experiments, outcomes = reeevaluation_results

thresholds = {'utility':0.75, 'inertia':0.99, 'reliability':0.8}

overall_scores = {}
for policy in experiments.policy.unique():
    logical = experiments.policy == policy
    scores = {}
    for k, v in outcomes.items():
        try:
            n = np.sum(v[logical]>=thresholds[k])
        except KeyError:
            continue
        scores[k] = n/1000 
    overall_scores[policy] = scores
        
overall_scores = pd.DataFrame(overall_scores).T

In [ ]:
limits = parcoords.get_limits(overall_scores)
paraxes = parcoords.ParallelAxes(limits)
paraxes.plot(overall_scores)
plt.show()

There are many ways of coding the regret calculating. Here I choose to use dataframes. I basically wanted to use the groupby method on a dataframe with the regret for each policy in each scenario. Reasoning backward from there, I arrived at the code shown below. 

There are probably ways of coding this even more efficient. My guess is that a groupby over the scenarios would help in calculating the best performance in each scnenario, avoiding one more explicit loop. I leave this as an exercise for you.

In [ ]:
overall_scores = {}
regret = []
for scenario in experiments.scenario.unique():
    logical = experiments.scenario==scenario
    temp_results = {k:v[logical] for k,v in outcomes.items()}
    temp_results = pd.DataFrame(temp_results)
    temp_experiments = experiments[experiments.scenario==scenario]
        
    best = temp_results.max()
    best['max_P'] = temp_results.max_P.min()
    scenario_regret = a - best
    scenario_regret['policy'] = temp_experiments.policy.values    
    regret.append(scenario_regret)


In [ ]:
regret = pd.concat(regret)
maxregret = regret.groupby('policy').max()

In [ ]:
limits = parcoords.get_limits(maxregret)
paraxes = parcoords.ParallelAxes(maxregret)
paraxes.plot(maxregret)
plt.show()